In [1]:
import numpy as np 
import matplotlib.pyplot as plt 
import json
import evaluate
import tqdm

/data/katie_kang/miniconda3/envs/trlx2/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2024-07-11 00:35:30,889] [INFO] [real_accelerator.py:191:get_accelerator] Setting ds_accelerator to cuda (auto detect)


In [2]:
rouge = evaluate.load('rouge')

In [5]:
train_num_correct = (np.concatenate([np.load("data/MATH_aug/train_aug_1_answer_types5_seed2.npy"), np.load("data/MATH_aug/train_aug_2_answer_types5_seed2.npy")], axis=0)==0).sum(axis=-1)
easy_train_idxs = np.where(train_num_correct==5)[0]
medium_train_idxs = np.where((train_num_correct<5)*(train_num_correct>0))[0]

In [4]:
with open('data/MATH_aug/AugMATH_part1.jsonl', 'r') as json_file:
    json_list = list(json_file)

with open('data/MATH_aug/AugMATH_part2.jsonl', 'r') as json_file:
    json_list += list(json_file)

train_questions = []
train_answers = []
for json_str in json_list:
    result = json.loads(json_str)
    train_questions.append(result["query"])
    train_answers.append(result["response"])
    
train_questions = np.array(train_questions)
train_answers = np.array(train_answers)

In [6]:
NUM_TRAIN_POINTS=20000
HARD_RATIO = 0.75
EASY_RATIO=(1-HARD_RATIO)*0.35
MEDIUM_RATIO=(1-HARD_RATIO)*0.65


CKPT_NAME=f"math_aug_easy{EASY_RATIO:.2f}_medium{MEDIUM_RATIO:.2f}_hard{HARD_RATIO:.2f}_total{NUM_TRAIN_POINTS}"

subsample_idxs = np.load("ckpts/"+CKPT_NAME+"/subsample_idxs.npy")[:5000]


rouges_hard0pt75 = []
ratios_hard0pt75 = []
for checkpoint_strs in ["2000", "4000", "6000", "8000"]:

    train_correct = np.load("ckpts/"+CKPT_NAME+"/checkpoint-"+checkpoint_strs+"/train_aug_subsample_answer_types5_seed2.npy")==0
    train_samples = np.load("ckpts/"+CKPT_NAME+"/checkpoint-"+checkpoint_strs+"/train_aug_subsample_answers5_seed2.npy")
    # hard_subsample = np.where(np.isin(subsample_idxs, hard_train_idxs))[0]
    easy_subsample = np.where(np.isin(subsample_idxs, easy_train_idxs))[0]


    incorrect = 0
    exact_match = 0
    correct_different = 0
    predictions =[]
    references = []
    for input_idx in easy_subsample:
        for sample_idx in range(5):
            correct = train_correct[input_idx, sample_idx]
            if correct:
                if train_samples[input_idx][sample_idx].strip() == train_answers[subsample_idxs[input_idx]].strip():
                    exact_match+=1
                else:
                    correct_different+=1
                predictions.append(train_samples[input_idx][sample_idx].strip())
                references.append(train_answers[subsample_idxs[input_idx]].strip())
            else:
                incorrect+=1
    # results = rouge.compute(predictions=predictions, references=references, use_aggregator=False)
    # rouges_hard0pt75.append(results['rougeL'])

    total = exact_match+correct_different+incorrect
    print(total)
    print(exact_match/total, correct_different/total, incorrect/total)
    ratios_hard0pt75.append([exact_match/total, correct_different/total, incorrect/total])

2210
0.006787330316742082 0.3755656108597285 0.6176470588235294
2210
0.02760180995475113 0.5782805429864254 0.3941176470588235
2210
0.15701357466063348 0.6085972850678733 0.2343891402714932
2210
0.4497737556561086 0.4502262443438914 0.1
